In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from PIL import Image
import yaml

In [ ]:
fn = 'videos/v1.mp4'
fn_yaml = 'yaml_data/v1.txt'
show_ids = True
loaded_model = tf.keras.models.load_model('empty_vs_occupied.h5')
cap = cv2.VideoCapture(fn)

def run_classifier(img, id):
    img = cv2.resize(img, (64, 64))
    img = Image.fromarray(img) # Returns an image object from array
    img = img.convert(mode='RGB') # convert to RGB 3-channel
    img = image.img_to_array(img, data_format = None)  # data uint8 not allowed in the list of allowed values
    img = np.expand_dims(img, axis = 0)  # add one more dimension
    result = loaded_model.predict(img) 
    if result[0][0]==1: # if empty
        return False
    else:
        return True 

In [ ]:
with open(fn_yaml, 'r') as stream:
    parking_data = yaml.load(stream)    
parking_bounding_rects = []
if parking_data != None:
    for park in parking_data:
        points = np.array(park['points'])
        rect = cv2.boundingRect(points)
        parking_bounding_rects.append(rect)

C:\Users\Lenovo\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [ ]:
i = 0
count_green = 0
while(cap.isOpened()):
    ret, frame_initial = cap.read()
    if ret == False :
        break
    if ret == True:
        i = i+1
        if i % 20 == 0:
            count_green = 0
            frame = cv2.resize(frame_initial, None, fx=2, fy=2)
            frame_blur = cv2.GaussianBlur(frame.copy(), (5,5), 3)
            frame_gray = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2GRAY)

            for ind, park in enumerate(parking_data):
                points = np.array(park['points'])
                rect = parking_bounding_rects[ind]
                roi_gray_ov = frame_gray[rect[1]:(rect[1] + rect[3]),
                                rect[0]:(rect[0] + rect[2])]  # crop roi for faster calcluation
                result = run_classifier(roi_gray_ov, ind)
                if result == False:
                    color = (0,255,0)
                    count_green = count_green + 1
                else:
                    color = (0,0,255)
                cv2.drawContours(frame, [points], contourIdx=-1, color=color, thickness=2, lineType=cv2.LINE_8) 
            cv2.putText(frame, 'No of available spots: ' +str(count_green), (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
            cv2.imshow('frame', frame)
           
    k = cv2.waitKey(1)
    if k == ord('q'):
        break 
                        
cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()